In [1]:
import sys
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from datatable import dt, f, by

from plotly import express as px, io as pio
pd.options.plotting.backend = 'plotly'
pio.renderers.default = 'plotly_mimetype+notebook_connected'

from utils import get_path, data_load
sys.path.insert(0, '../')
from secret import API_KEY

# Fontes de Dados

## Procedimentos

### Carregando tabela

In [2]:
path_proc = get_path('GESTANTES', 'procs_bienio.csv.gzip')
df_proc_bienio = pd.read_csv(path_proc)
df_proc_bienio

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,260
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,282
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3323
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3368
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167211,1,1,SL/SC/42016/421870,SL/SC/42007/420540/3157245,138.4773,104.940000,138.4773,104.940000,2,0_1,3_5,-28.4713,-49.0144,-27.5945,-48.5477,1
167212,1,1,SL/SC/42016/421870,SL/SC/42015/420460/2594277,54.6210,49.316667,54.6210,49.316667,2,0_0,3_4,-28.4713,-49.0144,-28.6723,-49.3729,1
167213,1,1,SL/SC/42016/421870,SL/SC/42016/420280/2665883,35.5439,37.493333,71.0878,74.986667,1,0_0,3_3,-28.4713,-49.0144,-28.2681,-49.1701,2
167214,1,1,SL/SC/42016/421870,SL/SC/42016/420730/2385880,53.9610,45.353333,53.9610,45.353333,1,0_0,3_3,-28.4713,-49.0144,-28.2284,-48.6659,1


## Localidades

### Carregando tabela

In [3]:
path_locs = get_path('LOCALIDADES', 'localidades.csv.gzip')
df_locs = pd.read_csv(path_locs)
df_locs

,codigo,cod_municipio,nome_municipio,capital,cod_regiao_saude,nome_regiao_saude,cod_uf,uf,nome_uf,sigla_regiao,nome_regiao,socioecon,latitude,longitude
0,CO/GO/52001/520005,520005,Abadia de Goiás,0,52001,Central,52,GO,Goiás,CO,Centro-Oeste,5,-16.75730,-49.4412
1,SD/MG/31052/310010,310010,Abadia dos Dourados,0,31052,Patrocínio Monte Carmelo,31,MG,Minas Gerais,SD,Sudeste,2,-18.48310,-47.3916
2,CO/GO/52011/520010,520010,Abadiânia,0,52011,Pirineus,52,GO,Goiás,CO,Centro-Oeste,3,-16.19700,-48.7057
3,SD/MG/31067/310020,310020,Abaeté,0,31067,Sete Lagoas,31,MG,Minas Gerais,SD,Sudeste,3,-19.15510,-45.4444
4,NT/PA/15011/150010,150010,Abaetetuba,0,15011,Tocantins,15,PA,Pará,NT,Norte,1,-1.72183,-48.8788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,ND/BA/29010/293360,293360,Xique-Xique,0,29010,Irecê,29,BA,Bahia,ND,Nordeste,1,-10.82300,-42.7245
5566,ND/PB/25005/251740,251740,Zabelê,0,25005,5ª Região,25,PB,Paraíba,ND,Nordeste,1,-8.07901,-37.1057
5567,SD/SP/35156/355715,355715,Zacarias,0,35156,José Bonifácio,35,SP,São Paulo,SD,Sudeste,3,-21.05060,-50.0552
5568,ND/MA/21019/211400,211400,Zé Doca,0,21019,Zé Doca,21,MA,Maranhão,ND,Nordeste,1,-3.27014,-45.6553


# Resultados

In [4]:
def select_and_group(df_proc, df_locs, cols, loc='nome_regiao', ref='origem'):
  cols_proc = cols.copy()
  cols_proc.insert(-1, ref)
  cols_locs = ['codigo', loc]
  df = df_proc[cols_proc].merge(df_locs[cols_locs], left_on=ref, right_on='codigo', how='left')
  cols_proc.insert(-1, loc)
  df = df[cols_proc]
  df = df.drop(ref, axis=1)
  df = df.groupby(by=list(df.columns)[:-1], as_index=False).sum()
  df['bienio'] = df['bienio'].map(
    {0: '2010-2011', 1: '2018-2019'})
  df['parto_normal'] = df['parto_normal'].map(
    {0: 'Parto Cesárea', 1: 'Parto Normal'})
  df = df.rename(columns={
    'bienio': 'Biênio',
    'parto_normal': 'Parto',
    'criticidade': 'Criticidade',
    loc: 'Localidade',
    'count': 'Procedimentos',
  })
  df = df.pivot_table('Procedimentos', list(df.columns)[1:-1], 'Biênio', sort=True)
  df = df.reset_index(level=df.index.names)
  df = pd.DataFrame(df.to_dict())
  return df

In [5]:
def fig_count_partos(df):
  fig = go.Figure()
  colors = ['#31A885','#DD6F1C']
  items = ['2010-2011','2018-2019']
  for item, color in zip(items, colors):
      fig.add_trace(
        go.Bar(
          x=df['Parto'], y=df[item],
          marker_color=color,
          text=df[item], textposition='auto',
          name=item,))
  buttons = []
  for loc in df['Localidade'].unique():
      buttons.append(dict(
        method='update',
        label=loc,
        args=[{
          'y': df[df['Localidade'] == loc][items].T.values
        }]))
  button_brasil = dict(
    method='update',
    label='Brasil',
    args=[{'y': df[items].T.values}])
  buttons = sorted(buttons, key=lambda d: d['label'])
  buttons.insert(0, button_brasil)
  updatemenu=[]
  your_menu=dict()
  updatemenu.append(your_menu)
  updatemenu[0]['buttons']=buttons
  updatemenu[0]['direction']='down'
  updatemenu[0]['showactive']=True
  fig.update_layout(
    title='Número total de partos realizados no SUS',
    legend_title='Biênio',
    updatemenus=updatemenu)
  return fig

## Partos no SUS por residência

In [6]:
def add_total(df):
  df_total = df[list(df.columns)[1:]]
  df_total = df_total.groupby(by='Localidade', as_index=False).sum()
  df_total['Parto'] = 'Ambos'
  df = pd.concat([df, df_total])
  df = df.sort_values(by=['Localidade', 'Parto'])
  df = df.reset_index(drop=True)
  return df

In [7]:
cols_proc = ['bienio', 'parto_normal', 'count']

### Região

In [8]:
df_regiao = add_total(select_and_group(df_proc_bienio, df_locs, cols_proc, loc='nome_regiao'))
fig_regiao = fig_count_partos(df_regiao)
fig_regiao.show()

### UF

In [9]:
df_uf = add_total(select_and_group(df_proc_bienio, df_locs, cols_proc, loc='nome_uf'))
fig_uf = fig_count_partos(df_uf)
fig_uf.show()

## Deslocamento % (município)

In [10]:
def critic(df, critic_level=1):
  df_critic = df[df['Criticidade'] >= critic_level - 1].copy()
  df_critic['Criticidade'] = df_critic['Criticidade'].apply(
    lambda x: x >= critic_level)
  df_critic = df_critic.groupby(
    by=list(df_critic.columns)[:-2],
    as_index=False
  ).sum()
  df_critic_num = df_critic[df_critic['Criticidade'] == True]
  df_critic_den = df_critic[df_critic['Criticidade'] == False]
  df_critic_num = df_critic_num.drop('Criticidade', axis=1)
  df_critic_den = df_critic_den.drop('Criticidade', axis=1)
  df_critic = pd.merge(
    df_critic_num, df_critic_den,
    how='left', on=list(df_critic_num.columns)[:-2],
    suffixes=['_num', '_den'])
  df_critic['2010-2011'] = round(df_critic['2010-2011_num'] / df_critic['2010-2011_den'] * 100, 2)
  df_critic['2018-2019'] = round(df_critic['2018-2019_num'] / df_critic['2018-2019_den'] * 100, 2)
  df_critic = df_critic.sort_values(by=['Localidade', 'Parto'])
  return df_critic

In [11]:
cols_critic = ['bienio', 'parto_normal', 'criticidade', 'count']

### Região

In [12]:
df_critic_regiao = critic(
  select_and_group(df_proc_bienio, df_locs, cols_critic),
  critic_level=1)
fig_critic_regiao = fig_count_partos(df_critic_regiao)
fig_critic_regiao.show()

### UF

In [13]:
df_critic_uf = critic(
  select_and_group(df_proc_bienio, df_locs, cols_critic, loc='nome_uf'),
  critic_level=1)
fig_critic_uf = fig_count_partos(df_critic_uf)
fig_critic_uf.show()

## Deslocamento % (região de saúde)

### Região

In [14]:
df_critic_regiao = critic(
  select_and_group(df_proc_bienio, df_locs, cols_critic),
  critic_level=2)
fig_critic_regiao = fig_count_partos(df_critic_regiao)
fig_critic_regiao.show()

### UF

In [15]:
df_critic_uf = critic(
  select_and_group(df_proc_bienio, df_locs, cols_critic, loc='nome_uf'),
  critic_level=2)
fig_critic_uf = fig_count_partos(df_critic_uf)
fig_critic_uf.show()